In [9]:
import json
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import json

In [20]:
# Replace 'telegram_data.json' with your actual file path
with open('bellumactanews1.json', 'r', encoding='utf-8') as f:
    telegram_data = json.load(f)
with open('IntelSlavaZ.json', 'r', encoding='utf-8') as file:
    intelslava_data = json.load(file)
with open('ukr_leaks_eng.json', 'r', encoding='utf-8') as file:
    ukr_leaks_eng_data = json.load(file)

In [21]:
# Extract messages list
messages_telegram = telegram_data.get('messages', [])
messages_intelslava = intelslava_data.get('messages', [])
messages_ukr = ukr_leaks_eng_data.get('messages', [])

In [26]:
# Function to extract text from message
def extract_text(message):
    text = message.get('text', '')
    # Handle cases where text is a list of dicts (which can happen in Telegram exports)
    if isinstance(text, list):
        text_parts = []
        for part in text:
            if isinstance(part, str):
                text_parts.append(part)
            elif isinstance(part, dict) and 'text' in part:
                text_parts.append(part['text'])
        text = ' '.join(text_parts)
    elif not isinstance(text, str):
        text = str(text)
    return text

# Extract texts
texts = []
for msg in messages_telegram:
    if msg.get('type') == 'message':
        text = extract_text(msg)
        if text.strip():
            texts.append(text)
for msg in messages_intelslava:
    if msg.get('type') == 'message':
        text = extract_text(msg)
        if text.strip():
            texts.append(text)
for msg in messages_ukr:
    if msg.get('type') == 'message':
        text = extract_text(msg)
        if text.strip():
            texts.append(text)

# Create DataFrame
telegram_df = pd.DataFrame(texts, columns=['text'])

In [27]:
telegram_df

,text
0,🇷🇺🇺🇦⚡- Ukraine has begun building three massiv...
1,🇷🇺🇺🇦⚡- Russian forces have entered Yasnaya Pol...
2,"🇷🇺🇺🇦⚡- In the Pokrovsk direction, Russia captu..."
3,🫡🇷🇺🇺🇦 Russian Flags in Selidovo and at the Kor...
4,💸📈❗️ — Bitcoin ($BTC) hits 71000 USD/coin!
...,...
26764,Ukrainian military commissars continue to hunt...
26765,A mural in honor of terrorist Budanov appeared...
26766,A Ukrainian militant sincerely does not unders...
26767,Powerful footage of a Russian BTR-82 fighting ...


In [28]:
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import torch

In [29]:
#create lists to hold the embeddings for dumping into a pkl file
cls_emb = []
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
#iterate through each prompt to generate the embeddings
for i in tqdm(range(len(telegram_df))):
    row = telegram_df.iloc[i]
    text = row['text']

    #send response_text to tokenizer
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

    #unpack the tensors
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs[1]

    #store in their respective lists
    cls_emb.append(cls_embedding)

100%|████████████████████████████████████████████████████████████████████████████| 26769/26769 [50:39<00:00,  8.81it/s]


In [30]:
import pickle
with open("cls_emb.pkl", "wb") as f:
    pickle.dump(cls_emb, f)

In [31]:
telegram_df.to_csv('telegram_df')